# Azure AI Foundry - Connected Agents

This notebook demonstrates how to create and orchestrate **3 connected agents** using Azure AI Foundry:

![](../media/img/ConnectedAgents.png)

1. **Orchestrator Agent** - Coordinates the other two agents to answer queries about sport events. 
2. **Sport Event Winner Agent** - Provides information about sport event winners
3. **Sport Event Results Agent** - Provides information about sport event results  

The orchestrator agent doesn't answer questions directly but instead connects to the specialized agents to gather information, showcasing the power of agent orchestration in Azure AI Foundry.

The **Sport Event Winner Agent** and **Sport Event Results Agent** just simulate retrieving scores and winners. Please add your functionality within the agent definition (Prompting, Tools, MCP ...)


## Step 1 - Load Environment

Two communicate with the Azure AI Foundry Agent Service two parameters are loaded from a configuration file. 

- foundryEndpoint: Endpoint of a created AI Foundry Project. 
- foundryModel: Deployed LLM model

The Foundry Project can be created using by deploying the [BICEP file 'setup.bicep'](../setup/setup.bicep) using the [../setup/setup.azcli - Azure CLI script](../setup/setup.azcli). Necessary parameter can be provided in the [../setup/parameters.json](../setup/parameters.json).






In [12]:
#r "nuget: DotNetEnv, 3.1.1"
#r "nuget: Azure.AI.Agents.Persistent, 1.0.0"
#r "nuget: Azure.Identity, 1.14.1"

using DotNetEnv;

string configurationFile = @"../config/config.env";
Env.Load(configurationFile);

string foundryEndpoint = Environment.GetEnvironmentVariable("SDK_FOUNDRY_ENDPOINT") ?? "Foundry endpoint not found";
string foundryModelDeployment = Environment.GetEnvironmentVariable("SDK_FOUNDRY_DEPLOYMENTNAME") ?? "Foundry model deployment not found";

Console.WriteLine("Configuration loaded... ");



Installed Packages Azure.AI.Agents.Persistent, 1.0.0 Azure.Identity, 1.14.1 DotNetEnv, 3.1.1

Configuration loaded... 


## Step 2 - Create PersistentAgentsClient

To authenticate against the Azure AI Foundry Agents Service 
- Azure CLI credentials or
- Visual Studio credentials
can be used.

Ensure that you're logged into Azure using e.g. the Azure CLI by executing `az login`



In [13]:
using Azure.Identity;
using Azure.AI.Agents.Persistent;

// Ensure that az login has been executed in the terminal
DefaultAzureCredentialOptions defaultAzureCredentialOptions = new DefaultAzureCredentialOptions
{
    ExcludeInteractiveBrowserCredential = true,
    ExcludeVisualStudioCredential = false,
    ExcludeEnvironmentCredential = true,
    ExcludeManagedIdentityCredential = true,
    ExcludeAzureCliCredential = false
};
DefaultAzureCredential defaultAzureCredential = new DefaultAzureCredential(defaultAzureCredentialOptions);

PersistentAgentsClient persistedAgentsClient = new PersistentAgentsClient(foundryEndpoint, defaultAzureCredential);


Console.WriteLine($"PersistentAgentsClient created ...");

PersistentAgentsClient created ...


## Step 3 - Define Agents

Three agents are defined: 

![](../media/img/ConnectedAgentsDetails.png)

where the `GetSportNewsOrchestrator` agent employes the connected agents `GetSportEventWinner` and `GetSportEventResults` to provide results. 

The `GetSportEventWinner` and `GetSportEventResults` angent simulate functionality to provide winner or results and will always provide the same results.

In [14]:
// Sport event winner agent
string systemMessage = @"
    You provide information about sport Event winners.
    You take the sport event and year as input.
    You provide always the same team name as response.
    The winner of the sport event you're asked is always 'Flying Munich Dolphins
";

PersistentAgent sportEventWinnerAgent = await persistedAgentsClient.Administration.CreateAgentAsync(
    model: foundryModelDeployment,
    name: "GetSportEventWinner",
    instructions: systemMessage,
    tools: [
    ]
);
ConnectedAgentToolDefinition connectedAgentSportEventWinner = new ConnectedAgentToolDefinition(
    new ConnectedAgentDetails(
        sportEventWinnerAgent.Id,
        sportEventWinnerAgent.Name,
        "Gets the winner of a sport event"
    )
);

// Sport event results agent
systemMessage = @"
    You provide information about sport Event results.
    You take the sport event and year as input.
    You pvoide always the same results.
    The results of the sport event you're asked is always '35:10
";
PersistentAgent sportEventResultsAgent = await persistedAgentsClient.Administration.CreateAgentAsync(
    model: foundryModelDeployment,
    name: "GetSportEventResults",
    instructions: systemMessage,
    tools: [
    ]
);
ConnectedAgentToolDefinition connectedAgentSportEventResult = new ConnectedAgentToolDefinition(
    new ConnectedAgentDetails(
        sportEventResultsAgent.Id,
        sportEventResultsAgent.Name,
        "Gets the results of a sport event"
    )
);

systemMessage = @"
    You orchestrate connected agents.
    You don't answer questions directly, but you connect to other agents to get the answers.
    You can connect to the 'GetSportEventWinner' agent to get information about winners of sport events.
    You can connect to the 'GetSportEventResults' agent to get information about results of sport events.
    Don't use your internal knowledge to answer questions.
";

//Orchestrator agent
PersistentAgent orchestratorAgent = await persistedAgentsClient.Administration.CreateAgentAsync(
    model: foundryModelDeployment,
    name: "GetSportNewsOrchestrator",
    instructions: systemMessage,
    tools: [
        connectedAgentSportEventWinner,
        connectedAgentSportEventResult
    ]
);

Console.WriteLine($"Orchestrator agent: {orchestratorAgent.Name} with ID {orchestratorAgent.Id}");
Console.WriteLine($"\tconnected agent: {sportEventResultsAgent.Name} with ID {sportEventResultsAgent.Id}");
Console.WriteLine($"\tconnected agent: {sportEventResultsAgent.Name} with ID {sportEventResultsAgent.Id}");
Console.WriteLine($"created ...");

Orchestrator agent: GetSportNewsOrchestrator with ID asst_JVrexHMrAGUgrLTYQXEKone2
	connected agent: GetSportEventResults with ID asst_qQlICDrSw2uuYLiMBvpJ3qhZ
	connected agent: GetSportEventResults with ID asst_qQlICDrSw2uuYLiMBvpJ3qhZ
created ...


## Step 4 - Create and Run Thread

A `PersistentAgentThread` is created with the ***Who won the Super Sports Championship 2025 and what was the result?*** initial user message.

A `ThreadRun' object is created connectiong the created Thread with the orchestrator agent.

In [15]:
PersistentAgentThread persistentAgentThread = await persistedAgentsClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await persistedAgentsClient.Messages.CreateMessageAsync(
    persistentAgentThread.Id,
    MessageRole.User,
    "Who won the Super Sports Championship 2025 and what was the result?");

// Run the agent
ThreadRun threadRun = await persistedAgentsClient.Runs.CreateRunAsync(persistentAgentThread, orchestratorAgent);

do
{
    await Task.Delay(500);
    threadRun = persistedAgentsClient.Runs.GetRun(persistentAgentThread.Id, threadRun.Id);
}

while (threadRun.Status == RunStatus.Queued
    || threadRun.Status == RunStatus.InProgress
    || threadRun.Status == RunStatus.RequiresAction);

Console.WriteLine($"Thread run {threadRun.Id} completed ...");

Thread run run_5O5tr7hRlZ4r0rP1gWp55FSv completed ...


## Step 5 - Retrieve Thread Messages

After finishing `ThreadRun` all thread messages are retrieved and shown.

In [16]:
using Azure; 

Pageable<PersistentThreadMessage> messages = persistedAgentsClient.Messages.GetMessages(
    threadId: persistentAgentThread.Id,
    order: ListSortOrder.Ascending
);

Console.WriteLine($"Messages in thread {persistentAgentThread.Id} ...");
foreach (PersistentThreadMessage threadMessage in messages)
{
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.WriteLine($"\tRole: '{threadMessage.Role}': {textItem.Text}");
        }
    }
}


Messages in thread thread_mahZawsJtVJTJnkPXDoYSeUK ...
	Role: 'user': Who won the Super Sports Championship 2025 and what was the result?


## Step 6 - Housekeeping

Created agents and threads are deleted.

In [17]:
persistedAgentsClient.Threads.DeleteThread(threadId: persistentAgentThread.Id);
persistedAgentsClient.Administration.DeleteAgent(agentId: orchestratorAgent.Id);
persistedAgentsClient.Administration.DeleteAgent(agentId: sportEventWinnerAgent.Id);
persistedAgentsClient.Administration.DeleteAgent(agentId: sportEventResultsAgent.Id);

Console.WriteLine("Agents and thread deleted ...");

Agents and thread deleted ...
